<a href="https://colab.research.google.com/github/Gail529/climate_change/blob/main/climate_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading the necessary libraries

In [1]:
!pip install praw
!pip install spacy

     |████████████████████████████████| 163kB 4.3MB/s 
     |████████████████████████████████| 71kB 5.1MB/s 


In [16]:
import os
import tweepy as tw
import praw
import pandas as pd
import datetime as dt
import spacy
nlp = spacy.load('en_core_web_sm')


import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import re
from nltk.tokenize import word_tokenize,sent_tokenize,TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
 



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# TWEEPY for tweets

In [5]:
ACCESS_TOKEN="1118886346054090753-Vfsmdr6V8bKfKZS0uztpSzM3XR8LX0"
ACCESS_SECRET="8nyqL6bXe5yNd7kph55abUaaXn7qkEncrXxyi2ZcmOQPi"
CONSUMER_KEY="LpqLxpB3zzC8VZ3xPUmT75uU9"
CONSUMER_SECRET="qExPJvsm5VO4PLm1gmWQDyq7U6bjIuS7jT1aTpRdXQ3mWHCyn8"



def connect_to_twitter_OAuth():
    auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tw.API(auth, wait_on_rate_limit=True)
    return api
 
 
api = connect_to_twitter_OAuth()

In [6]:
search_words = " #COP21 OR #alarmism OR #climat OR #cities4climate OR #climateaction OR #climatechange OR #gmo OR #renewableenergy OR #earthtweet  OR #foodjustice OR #sustainable OR #4change OR #cleantech OR #windenergy OR #globalwarming OR #warmism OR #parispledgeofaction "
search_terms = search_words + " -filter:retweets"
date_since = "2020-03-01"
 
 
tweets = tw.Cursor(api.search,
              q=search_terms,
              lang="en",
              since=date_since).items(2000)

In [7]:
users_tweets = [[tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]
tweet_df1 = pd.DataFrame(data=users_tweets, 
                    columns=['user', "location","text"])

In [9]:
tweet_df1.to_csv('climate_tweet_df.csv')
tweet_df1.head(2)

,user,location,text
0,KVajpayaee,,@CathyFur @GailHepburn @bnorm_al @Crystalberry...
1,smidorima22_83,,Sponsored post:\nThinking about switching to #...
2,elemenous,"Northbrook, IL",Read the newest edition of the Global Educatio...
3,GeraldKutney,"Ottawa, Ontario","As pressure builds on oil and gas companies, s..."
4,Josephusssss,Somewhereshire,Remember when we used to put lead in our fuel?...


# PRAW for reddit

In [ ]:
reddit = praw.Reddit(client_id="CF806mHKsiEsjw", \
                     client_secret="urT2z76kylH16FVJCv69lhsmdD81dw", \
                     user_agent="#" 
                     )


In [ ]:
topics_dict = {
    "title":[], 
    "score":[], 
    "id":[],
    "url":[], 
    "comms_num": [], 
    "created": [], 
    "body":[]
    }


subred='climate change+emission trading+carbon tax+alarmism+global warming+climate change+emmissions+environmental defence+EPA+climate change'

subreddit = reddit.subreddit(subred)
top_subreddit = subreddit.hot(limit=500)

for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

topics_data = pd.DataFrame(topics_dict)
topics_data
topics_data.to_csv('reddit_Df.csv', index=False)


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
topics_data

In [ ]:
data=topics_data
text=data['title'].values
print(text)

# Text preprocessing

In [12]:
tweets=tweet_df1['text'].values
print(len(tweets))

2000


In [13]:
#lowercasing and url,punctuations and numbers removal,
def Lowercasing(words):
    string=re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "",str(words))
    word=string.lower()
    return word

#Tokenization and (@)handle extraction
def Tokenization(tweet):
    tokenizer=TweetTokenizer(strip_handles=True)
    tokens = tokenizer.tokenize(tweet)
    return tokens

#punctuations
def Punctuation_removal(tokens):
    words=[ word for word in tokens if word.isalnum()]
    return words

#stemming
def stemming(text):
    stemmer=PorterStemmer()
    for  word in text:
        stemmed_words=stemmer.stem(word)
        return stemmed_words

#stopword_removal
def remove_stopwords(words):
    stop_words=set(stopwords.words("english")) 
    result=[word for word in words if word not in stop_words ]
    return result


#lemmatization
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    lemmatized_phrase=[]
    for word in text:
        lemmatized_word=lemmatizer.lemmatize(word)
        lemmatized_phrase.append(lemmatized_word)
    return lemmatized_phrase
 

In [17]:
tokenized_tweets=[]
final_tweet=[]
    #tokenization , (@)handle removal and lemmatization
for i in range(0,1999):
    tweet=tweets[i]
    tweet_tokens=Tokenization(tweet)
    lemmatized_tweet=lemmatization(tweet_tokens)#lemmatization
    tokenized_tweets.append(lemmatized_tweet)

#lowercasing url extraction and punctuation removal
for i in tokenized_tweets:
    lowercased_string=Lowercasing(i)#lowercasing and removing numbers
    final_tweet.append(lowercased_string)


In [18]:
final_tweet

[' ',
 'sponsored post  thinking about switching to renewableenergy  start here   these other secto  ',
 'read the newest edition of the global education report  globaled globaled21  content from  ',
 'as pressure build on oil and gas company  some major player are making big climate pledge  we should weigh the  ',
 'remember when we used to put lead in our fuel   that lead wa released into the atmosphere for all to enjoy  we a  ',
 'aquagoat is a project that help save our  ',
 'five polish citizen are taking their government to court over it failure to protect them from the impact of the  ',
 'on her way out  merkel could set stage for germany a even more progressive climate partner  ',
 'climateaction so much smarter and futurethinking than the approach of the emissions reduction minister   ',
 'senators challenge to quit his  275k  yr post if he fails to meet climatechange target   thats just  ',
 'a suggestion to you for your big idea to change the world  fit solar panel on all ne